In [1]:
from __future__ import print_function
import cv2 as cv
import argparse
import numpy as np


## [capture]
capture = cv.VideoCapture('videos/MVI_0046.MOV')
if not capture.isOpened():
    print('Unable to open')
    exit(0)

backSub = cv.createBackgroundSubtractorMOG2(detectShadows = True)


#Custom kernel
kernel = None


while True:
    ret, frame = capture.read()
    if frame is None:
        break

    #Apply background object on the fram to get the segmented mask.
    fgMask = backSub.apply(frame)
    
    #May remove  
    #////////////////////////////
    #update the background model
    #fgMask = backSub.apply(frame)
    #////////////////////////////

    #Removes shadows
    _, fgMask = cv.threshold(fgMask, 250, 250, cv.THRESH_BINARY)

    #Applying morphological operations to produce a good mask
    fgMask = cv.erode(fgMask, kernel, iterations=1)
    fgMask = cv.dilate(fgMask, kernel, iterations=2)

    #Detect contours in the frame
    contours, _ = cv.findContours(fgMask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    #Create a copy of the frame to draw boundary on dice
    frameCopy = frame.copy()

    #Note
    #/////
    # Need to fix the rectangle that is retrieved from the contour. It isn't tracking correctly
    #/////
    for cnt in contours:

        if cv.contourArea(cnt) > 200 and cv.contourArea(cnt) < 400:

            #Retrieves the rectangle cordinates from the contour
            x, y, width, height = cv.boundingRect(cnt)

            #Draw Rectangle around dice
            cv.rectangle(frameCopy, (x, y), (x + width, y + height), (0,0,255), 2)

            # ERROR CHECKING
            # ////
            # cv.putText(frameCopy, 'Di Detected', (x, y-10), cv.FONT_HERSHEY_SIMPLEX, 0.3, (0,255,0), 1, cv.LINE_AA)
            # ////
    
    ## [display_frame_number]
    #get the frame number and write it on the current frame
    cv.rectangle(frame, (10, 2), (100,20), (255,255,255), -1)
    cv.putText(frame, str(capture.get(cv.CAP_PROP_POS_FRAMES)), (15, 15),
               cv.FONT_HERSHEY_SIMPLEX, 0.5 , (0,0,0))

    # Draw ROI rectangle
    #cv.rectangle(frame, (roi_x, roi_y), (roi_x + roi_width, roi_y + roi_height), (0, 255, 0), 2)

    #Extract fg from the frame using the segmented mask
    fgSection = cv.bitwise_and(frame, frame, mask=fgMask)

    stack = np.hstack((frame, fgSection, frameCopy))
    
    ## [show]
    #show the current frame and the fg masks
    #cv.imshow('Frame', frame)
    #cv.imshow('FG Mask', fgMask)
    cv.imshow("OG Frame, Extracted fg", cv.resize(stack, None, fx=0.5, fy=0.5))

    
    keyboard = cv.waitKey(30)
    if keyboard == 'q' or keyboard == 27:
        break

    #Quit key
    if keyboard == ord('h'):
        break

# Release the video capture object and close all windows
capture.release()
cv.destroyAllWindows()